In [ ]:
import cv2
import numpy as np
import os
import random
import matplotlib.pyplot as plt

def show_images(original, transformed, title="Ảnh sau biến đổi"):
    """Hiển thị song song ảnh gốc và ảnh đã biến đổi"""
    plt.figure(figsize=(8, 4))
    
    # Ảnh gốc
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title("Ảnh gốc")
    plt.axis("off")
    
    # Ảnh sau biến đổi
    plt.subplot(1, 2, 2)
    cmap = 'gray' if len(transformed.shape) == 2 else None
    plt.imshow(transformed, cmap=cmap)
    plt.title(title)
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

# Các hàm biến đổi ảnh
def inverse_transform(image):
    return cv2.bitwise_not(image)

def gamma_correction(image, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def log_transformation(image):
    image = image.astype(np.float32)
    c = 255 / np.log(1 + np.max(image))
    return (c * (np.log(image + 1))).astype(np.uint8)

def histogram_equalization(image):
    if len(image.shape) == 2:
        return cv2.equalizeHist(image)
    else:
        channels = cv2.split(image)
        eq_channels = [cv2.equalizeHist(ch) for ch in channels]
        return cv2.merge(eq_channels)

def contrast_stretching(image):
    min_val, max_val = np.min(image), np.max(image)
    return ((image - min_val) * (255 / (max_val - min_val))).astype(np.uint8)

# Xáo trộn kênh RGB
def shuffle_rgb(image):
    channels = list(cv2.split(image))
    random.shuffle(channels)
    return cv2.merge(channels)

# Xử lý tất cả ảnh trong folder
def process_images(output_folder):
    input_folder = "exercise"
    os.makedirs(output_folder, exist_ok=True)

    transformations = [
        ("Inverse Transform", inverse_transform),
        ("Gamma Correction", gamma_correction),
        ("Log Transform", log_transformation),
        ("Histogram Equalization", histogram_equalization),
        ("Contrast Stretching", contrast_stretching)
    ]

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            shuffled_image = shuffle_rgb(image)
            transform_name, transform_func = random.choice(transformations)
            grayscale_image = cv2.cvtColor(shuffled_image, cv2.COLOR_BGR2GRAY)
            transformed_image = transform_func(grayscale_image)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, transformed_image)

            print(f"Đã xử lý: {filename} - {transform_name}")
            show_images(image, transformed_image, f"{transform_name}: {filename}")
            
process_images("output_transformed")
